# Construct a DF that contains basically everything

In [1]:
import sys
if '..' not in sys.path: sys.path.append('..')

import pandas as pd
from utils import Adni, misc_util
import numpy as np

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
adni1_input_dir = '../data/ADNI1/SPM_preprocessed_normalized'

In [4]:
base_dir = '../data/ADNI1'
adni = Adni(root=misc_util.absolute_path(base_dir) + '/', processed=True)
adni.set_meta('../data/meta')

In [5]:
adni.load(show_output=False)
df = adni.to_df(show_output=False)

('C:\\Users\\Erik\\Dev\\HH\\Master\\Git\\Master-thesis\\Development\\data\\ADNI1/SPM_preprocessed_normalized/', ['projectIdentifier', 'subject.subjectIdentifier', 'subject.study.series.modality', 'subject.study.imagingProtocol.description', 'subject.study.series.dateAcquiredPrecise', 'image_nbr', 'series', 'subject.study.imagingProtocol.imageUID', 'filename', 'path'], <bound method Adni.info_from_raw_filename of <utils.adni.Adni object at 0x000001F748A57DF0>>)


In [6]:
df

,projectIdentifier_x,subject.subjectIdentifier,subject.study.series.modality_x,subject.study.imagingProtocol.description_x,subject.study.series.dateAcquiredPrecise,image_nbr,series,subject.study.imagingProtocol.imageUID,filename,path,...,subject.study.imagingProtocol.protocolTerm.protocol_Decay Correction,subject.study.imagingProtocol.protocolTerm.protocol_Reconstruction,subject.study.imagingProtocol.protocolTerm.protocol_Scatter Correction,subject.study.imagingProtocol.protocolTerm.protocol_Radioisotope,subject.study.imagingProtocol.protocolTerm.protocol_Convolution Kernel,subject.visit.assessment.component.assessmentScore_MMSCORE,subject.visit.assessment.component.assessmentScore_GDTOTAL,subject.visit.assessment.component.assessmentScore_CDGLOBAL,subject.visit.assessment.component.assessmentScore_NPISCORE,subject.visit.assessment.component.assessmentScore_FAQTOTAL
0,ADNI,003_S_1057,PT,PET_ADNI,20100203143109531,74,S79703,I165739,iwADNI_003_S_1057_PT_PET_ADNI_br_raw_201002031...,C:\Users\Erik\Dev\HH\Master\Git\Master-thesis\...,...,START,OSEM2D 4i16s,Model-based,F-18,All-pass,23.0,0.0,1.0,6.0,15.0
1,ADNI,003_S_1059,PT,PET_ADNI,20090106115424676,29,S61310,I132791,iwADNI_003_S_1059_PT_PET_ADNI_br_raw_200901061...,C:\Users\Erik\Dev\HH\Master\Git\Master-thesis\...,...,START,OSEM2D 4i16s,Model-based,F-18,All-pass,24.0,10.0,1.0,0.0,10.0
2,ADNI,003_S_1074,PT,ADNI__AC__,20100420112336966,431,S83424,I171424,iwADNI_003_S_1074_PT_ADNI__AC___br_raw_2010042...,C:\Users\Erik\Dev\HH\Master\Git\Master-thesis\...,...,START,OSEM2D 4i16s,Model-based,F-18,All-pass,27.0,1.0,0.0,0.0,2.0
3,ADNI,003_S_1122,PT,PET_ADNI,20100203142619359,81,S79702,I165738,iwADNI_003_S_1122_PT_PET_ADNI_br_raw_201002031...,C:\Users\Erik\Dev\HH\Master\Git\Master-thesis\...,...,START,OSEM2D 4i16s,Model-based,F-18,All-pass,30.0,3.0,0.5,3.0,7.0
4,ADNI,003_S_1257,PT,ADNI,20070927124534984,1,S40125,I75578,iwADNI_003_S_1257_PT_ADNI_br_raw_2007092712453...,C:\Users\Erik\Dev\HH\Master\Git\Master-thesis\...,...,START,OSEM 6i16s,Model-based,F-18,NaN,20.0,NaN,2.0,4.0,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401,ADNI,941_S_1197,PT,ADNI_Brain_PET__Raw,20090226090220703,540,S63625,I137082,iwADNI_941_S_1197_PT_ADNI_Brain_PET__Raw_br_ra...,C:\Users\Erik\Dev\HH\Master\Git\Master-thesis\...,...,START,LOR-RAMLA,SS-SIMUL,F-18,NaN,30.0,0.0,0.0,0.0,0.0
402,ADNI,941_S_1202,PT,ADNI_Brain_PET__Raw,20100318104904874,440,S81232,I168059,iwADNI_941_S_1202_PT_ADNI_Brain_PET__Raw_br_ra...,C:\Users\Erik\Dev\HH\Master\Git\Master-thesis\...,...,START,LOR-RAMLA,SS-SIMUL,F-18,NaN,30.0,0.0,0.5,9.0,10.0
403,ADNI,941_S_1203,PT,ADNI_Brain_PET__Raw,20100326154019681,509,S82369,I169591,iwADNI_941_S_1203_PT_ADNI_Brain_PET__Raw_br_ra...,C:\Users\Erik\Dev\HH\Master\Git\Master-thesis\...,...,START,LOR-RAMLA,SS-SIMUL,F-18,NaN,29.0,0.0,0.5,0.0,2.0
404,ADNI,941_S_1295,PT,ADNI_Brain_PET__Raw,20080825150245948,540,S55789,I116815,iwADNI_941_S_1295_PT_ADNI_Brain_PET__Raw_br_ra...,C:\Users\Erik\Dev\HH\Master\Git\Master-thesis\...,...,START,LOR-RAMLA,SS-SIMUL,F-18,NaN,25.0,NaN,0.5,9.0,5.0


## Add RID and site key

In [7]:
df['RID'] = df.apply(lambda row: int(row['subject.subjectIdentifier'].split('_')[-1]), axis=1)
df['Site'] = df.apply(lambda row: int(row['subject.subjectIdentifier'].split('_')[0]), axis=1)

## Add MCI progression

In [8]:
mci_progression_data = pd.read_csv('../data/misc/ADNI1/MCI_progression.csv')

In [9]:
def mci_conversion(row):
    if row['subject.researchGroup'] == 'MCI':
        if row['RID'] in mci_progression_data['RID']:
            return True
        else:
            return False
    else:
        return None

df['MCI_progression'] = df.apply(mci_conversion, axis = 1)

## Impute missing MMSE data

In [10]:
mmse_detailed_data = pd.read_csv('../data/misc/MMSE.csv', parse_dates = ['EXAMDATE'], low_memory = False)

In [11]:
def retrieve_MMSE(row):
    if not pd.isna(row['subject.visit.assessment.component.assessmentScore_MMSCORE']):
        return row['subject.visit.assessment.component.assessmentScore_MMSCORE']
    scores = []
    differences = []
    for index, MMSErow in mmse_detailed_data.loc[(mmse_detailed_data['EXAMDATE'].isna() == False) & (mmse_detailed_data['RID'] == row['RID'])].iterrows():
        scores.append(MMSErow['MMSCORE'])
        differences.append(abs((row['subject.study.series.dateAcquired'] - MMSErow['EXAMDATE']).total_seconds()))
    return scores[np.argmin(differences)]

In [12]:
df['Imputed_MMSE'] = df.apply(retrieve_MMSE, axis = 1)